In [1]:
import numba

In [4]:
import numpy as np

import accumulator as Accum
import accumulator2 as Accum2

from importlib import reload

In [5]:
reload(Accum)
reload(Accum2)

<module 'accumulator2' from '/Users/wpk/Documents/python/projects/python_dropin_files/accumulator2.py'>

In [6]:
x = np.random.rand(1000)

data = np.zeros(3)
for xx in x:
    Accum._push_stat(data, 1.0, xx, 0.0)

np.testing.assert_allclose(data[1] , x.mean())
np.testing.assert_allclose(data[2] , x.var(ddof=0))

idx = np.cumsum(np.random.randint(1, 20, 1000))
idx = idx[idx < len(x)]
X = np.split(x, idx)

A = [xx.mean() for xx in X]
V = [xx.var() for xx in X]
W = [len(xx) for xx in X]

data = np.zeros(3)
L = []
for a, v, w in zip(A, V, W):
    Accum._push_stat(data, a=a, v=v, w=w)
    L.append(data)

np.testing.assert_allclose(data[1], x.mean())
np.testing.assert_allclose(data[2], x.var(ddof=0))

L = np.array(L)

In [10]:
%%timeit -n 1
g = Accum.StatsAccum.from_stats(np.array(A), np.array(V), np.array(W))

np.testing.assert_allclose(g.mean(), x.mean())
np.testing.assert_allclose(g.var(), x.var(ddof=0))
np.testing.assert_allclose(g.weight(), len(x))


520 µs ± 116 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit -n 1
g = Accum2.StatsAccum.from_stats(np.array(A), np.array(V), np.array(W))

np.testing.assert_allclose(g.mean(), x.mean())
np.testing.assert_allclose(g.var(), x.var(ddof=0))
np.testing.assert_allclose(g.weight(), len(x))


481 µs ± 135 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
x = np.random.rand(5000, 100)


idx = np.cumsum(np.random.randint(1, 20, 1000))
idx = idx[idx < len(x)]
X = np.split(x, idx)

LA = [xx.mean(axis=0) for xx in X]
LV = [xx.var(axis=0, ddof=0) for xx in X]
LW = [xx.shape[0] for xx in X]

LA = np.array(LA)
LV = np.array(LV)
LW = np.array(LW)



s = (slice(None),) + (None,) * (x.ndim - 1)
LW = np.broadcast_to(LW[s], LA.shape)

In [15]:
g = Accum.StatsAccumVec.from_stats(A=LA, V=LV, W=LW)

np.testing.assert_allclose(g.mean(), x.mean(axis=0))
np.testing.assert_allclose(g.var(), x.var(axis=0, ddof=0))
np.testing.assert_allclose(g.weight(), np.count_nonzero(x, axis=0))


g = Accum2.StatsAccumVec.from_stats(A=LA, V=LV, W=LW)

np.testing.assert_allclose(g.mean(), x.mean(axis=0))
np.testing.assert_allclose(g.var(), x.var(axis=0, ddof=0))
np.testing.assert_allclose(g.weight(), np.count_nonzero(x, axis=0))

In [20]:
%timeit -n 10 -r 3 Accum.StatsAccumVec.from_stats(A=LA, V=LV, W=LW)
%timeit -n 10 -r 3 Accum2.StatsAccumVec.from_stats(A=LA, V=LV, W=LW)


998 µs ± 27.1 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
770 µs ± 48.5 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [21]:
# by value 

In [22]:
X = np.random.rand(10000, 100)
W = np.random.rand(10000, 1)
#W = np.ones((1000, 1), dtype=X.dtype)
W = np.broadcast_to(W, X.shape)

In [23]:
g = Accum.StatsAccumVec.from_vals(X, W=W)

In [24]:
data = np.zeros_like(g._datar)

In [25]:
data.fill(0.0)
Accum._push_vals_vec(data, W, X)
np.testing.assert_allclose(data, g.data)

In [26]:
data.fill(0.0)
Accum2._push_vals_vec(data, W, X)
np.testing.assert_allclose(data, g.data)

In [30]:
%timeit -n 10 -r 3 Accum._push_vals_vec(data, W, X)
%timeit -n 10 -r 3 Accum2._push_vals_vec(data, W, X)


13.3 ms ± 964 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)
2.45 ms ± 49.9 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [41]:
# numba
from numba import njit, prange, jit

_PARALLEL = True
@njit(parallel=True)
def _push_vals_vec(data, W, X):
    nmeta = data.shape[0]
    nsamp = X.shape[0]

    for s in prange(nsamp):
        for m in range(nmeta):
            _w = W[s,m]
            if _w == 0.0:
                return

            data[m, 0] += _w
            alpha = _w / data[m, 0]

            delta = (X[s,m] - data[m, 1])
            incr = delta * alpha

            data[m, 1] += incr
            data[m, 2] += (1.0 - alpha) * (delta * incr) - alpha * data[m, 2]


In [44]:
data.fill(0.0)
_push_vals_vec(data, W, X)
np.testing.assert_allclose(data, g.data)

%timeit -n 10 -r 3  _push_vals_vec(data, W, X)


3.02 ms ± 340 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [39]:
data.fill(0.0)
_push_vals_vec(data, W, X)
np.testing.assert_allclose(data, g.data)

%timeit -n 10 -r 3  _push_vals_vec(data, W, X)


2.86 ms ± 176 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [240]:
_push_vals_vec.py_func

CPUDispatcher(<function _push_vals_vec at 0x615dd16a8>)

In [235]:
# @njit
# def _push_val(data, w, x):
#     if w == 0.0:
#         return

#     data[0] += w
#     alpha = w / data[0]

#     delta = (x - data[1])
#     incr = delta * alpha

#     data[1] += incr
#     data[2] += (1.0 - alpha) * (delta * incr) - alpha * data[2]
    
@njit
def _push_val_inc(data, w, x):
    if w == 0.0:
        return

    alpha = w / (data[0] + w)

    delta = (x - data[1])
    incr = delta * alpha
    
    out = np.zeros(3, dtype=data.dtype)
    out[0] = w
    out[1] = incr
    out[2] = (1.0 - alpha) * (delta * incr) - alpha * data[2]
    return out
    
_PARALLEL = False
#@njit(parallel=_PARALLEL)
@jit
def _push_vals_vec2(data, W, X):
    nmeta = data.shape[0]
    nsamp = X.shape[0]

    for s in range(nsamp):
        for m in range(nmeta):
            out = _push_val_inc(data[m, :], W[s, m], X[s, m])
            data[m,:] += out #_push_val_inc(data[m, :], W[s, m], X[s, m])


In [237]:
data.fill(0.0)
_push_vals_vec2(data, W, X)

In [232]:
a[0, :] += b

In [219]:
np.testing.assert_allclose(data, g.data)

#%timeit -n 10 -r 3  _push_vals_vec2(data, W, X)

In [45]:
# random data
Data = np.random.rand(100, 5, 3)

In [55]:
g = Accum.StatsAccumVec.from_datas(Data=Data, axis=0)

In [56]:
data = np.zeros_like(g._datar)
Accum._push_stats_data_vec(data, Data)

In [59]:
np.testing.assert_allclose(data, g.data)